In [1]:
# %%
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import polars as pl
import pandas as pd
import numpy as np
import json
import seaborn as sns

sns.set()

from catboost import CatBoostRegressor
REGRESSION_TARGET = "prix"
CLASSIFICATION_TARGET = "en_dessous_du_marche"
random_state = 42

In [2]:
# Changer l'adresse du fichier si besoin
transactions = pl.read_parquet( "transactions_post_feature_engineering.parquet")


In [3]:
X = transactions.drop([REGRESSION_TARGET, CLASSIFICATION_TARGET]).to_pandas()
y_regression = transactions[REGRESSION_TARGET].to_pandas()

In [4]:
# Changer l'adresse des fichiers si beosin
with open("/features_used.json", "r") as f:
    feature_names = json.load(f)

with open("/categorical_features_used.json", "r") as f:
    categorical_features = json.load(f)

numerical_features = [col for col in feature_names if col not in categorical_features]

In [5]:
catboost_regressor = CatBoostRegressor(random_state=random_state, verbose=False)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_regression, random_state=random_state
)

In [8]:
catboost_regressor.fit(X_train[feature_names], y_train)

In [9]:
y_pred_train = catboost_regressor.predict(X_train)
errors = y_train - y_pred_train    
X_train["error"] = errors

y_pred_test = catboost_regressor.predict(X_test)
errors = y_test - y_pred_test    
X_test["error"] = errors